In [92]:
import pandas as pd

In [93]:
import pandas as pd

# First, processing and preparing the emissions dataset:

# load data
df = pd.read_csv('/Users/jamiemccay/Desktop/Personal/Projects/OuterHeaven/.vscode/data/annual_air_emissions.csv')

# filter data 
df = df[df['Gas_Type'] == 'Carbon dioxide']
cols_to_select = ['F{}'.format(year) for year in range(2010, 2022)]
df = df[cols_to_select]

# generate output
output = {
    'labels': [],
    'values': []
}
for col in df.columns.tolist():
    output['labels'].append(col[1:])
    output['values'].append(df[col].sum())
    
print(output)

# Next, the energy transition dataset:

# load data
df = pd.read_csv('/Users/jamiemccay/Desktop/Personal/Projects/OuterHeaven/.vscode/data/energy_transition.csv')
# df.head()

# filter data
renewable_df = df[df['Energy_Type'] == "Total Renewable"]
non_renewable_df = df[df['Energy_Type'] == "Total Non-Renewable"]

cols_to_select = ['F{}'.format(year) for year in range(2000, 2021)]
renewable_df = renewable_df[cols_to_select]
non_renewable_df = non_renewable_df[cols_to_select]

# generate output
output = {
    "renewable": {
        'labels': [],
        'values': []
    },
    "non_renewable": {
        'labels': [],
        'values': []
    },
}

for col in renewable_df.columns.tolist():
    output['renewable']['labels'].append(col[1:])
    output['renewable']['values'].append(df[col].sum())
    
for col in non_renewable_df.columns.tolist():
    output['non_renewable']['labels'].append(col[1:])
    output['non_renewable']['values'].append(df[col].sum())

print(output)

# Next, process the stats on UK household waste:

# load dataset
df = pd.read_csv("/Users/jamiemccay/Desktop/Personal/Projects/OuterHeaven/.vscode/data/uk_waste_stats.csv")
# df.head()

# prepare data
df = df[df['Measure'] == 'Recycling rate']
df = df[['Year','Measure','UK']]
df.head()

# generate output
output = {
    'recycling_rate': {
        'labels': [],
        'values': []
    }
}
for index, row in df.iterrows():
    output['recycling_rate']['labels'].append(row['Year'])
    output['recycling_rate']['values'].append(row['UK'])
    
print(output)




{'labels': ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'], 'values': [347281.108599036, 357164.129203444, 362576.51395351195, 367143.053957388, 370164.63728685095, 368087.792373102, 367358.274453194, 373584.65754200297, 381289.436725764, 381536.116002269, 361909.968731441, 380874.15337320295]}
{'renewable': {'labels': ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'], 'values': [120257439.2143567, 121250602.571417, 125835765.27341701, 130206193.17641701, 135760891.9123047, 141158011.3425107, 146349423.4272768, 152461076.0733458, 155867326.8988108, 156089953.04070678, 165781600.4476386, 171199248.54092962, 175542736.9831332, 180759547.0959541, 184764000.18295872, 188540021.9097914, 193990067.31953448, 199651070.2852374, 207063949.5988404, 210314667.7598194, 212490440.76309842]}, 'non_renewable': {'labels': ['2000', '2001

In [94]:
# load the dataset
df = pd.read_csv('/Users/jamiemccay/Desktop/Personal/Projects/OuterHeaven/.vscode/data/processed_ni_waste_data.csv')

# initial pre-processing
df.rename(columns={'AreaName': 'area'}, inplace=True)
df.fillna(0, inplace=True)
df.replace('-', 0, inplace=True)

str_cols = ['quarter','quarter_name','fin_year','area']
for col in df.columns.tolist():
    if col in str_cols:
        pass
    else:
        df[col] = df[col].apply(lambda x: str(x).replace(',',''))
        df[col] = df[col].astype('float')

# sum up quarters to get annual totals instead of quarterly
df = df.groupby(['fin_year','area']).agg({
        'la_total_waste': 'sum',
        'la_reuse': 'sum',
        'la_dry_recycle': 'sum',
        'la_compost': 'sum',
        'la_energy_recovery_mixed_residual': 'sum',
        'la_energy_recovery_specific_stream': 'sum',
        'la_landfill': 'sum',
        'la_biodegradable_to_landfill': 'sum',
        'hh_total_waste': 'sum',
        'hh_reuse': 'sum',
        'hh_dry_recycle': 'sum',
        'hh_compost': 'sum',
        'hh_landfill': 'sum',
        'population': 'mean',
        'num_households': 'mean'
    }).reset_index()

# show sample of data
df.head()

,fin_year,area,la_total_waste,la_reuse,la_dry_recycle,la_compost,la_energy_recovery_mixed_residual,la_energy_recovery_specific_stream,la_landfill,la_biodegradable_to_landfill,hh_total_waste,hh_reuse,hh_dry_recycle,hh_compost,hh_landfill,population,num_households
0,2006/07,Antrim,36791.0,0.0,8984.0,8352.0,0.0,0.0,19414.0,0.0,32732.0,0.0,7069.0,8352.0,17268.0,51059.0,0.0
1,2006/07,Ards,48673.0,0.0,5267.0,6661.0,0.0,0.0,36745.0,0.0,45592.0,0.0,4944.0,6661.0,33986.0,75767.0,0.0
2,2006/07,Armagh,29198.0,0.0,6330.0,3878.0,0.0,0.0,18989.0,0.0,25825.0,0.0,4599.0,3878.0,17346.0,56785.0,0.0
3,2006/07,Ballymena,38269.0,0.0,5520.0,4612.0,0.0,0.0,28137.0,0.0,35989.0,0.0,5520.0,4612.0,25856.0,61447.0,0.0
4,2006/07,Ballymoney,15923.0,0.0,2152.0,1341.0,0.0,0.0,12373.0,0.0,14115.0,0.0,2152.0,1341.0,10621.0,29392.0,0.0


In [95]:
# local authority (la) waste dataset
la_df = df[['fin_year','area','la_total_waste','la_reuse','la_dry_recycle','la_compost','la_energy_recovery_mixed_residual','la_energy_recovery_specific_stream','la_landfill','la_biodegradable_to_landfill','num_households','population']]
la_df.head()

,fin_year,area,la_total_waste,la_reuse,la_dry_recycle,la_compost,la_energy_recovery_mixed_residual,la_energy_recovery_specific_stream,la_landfill,la_biodegradable_to_landfill,num_households,population
0,2006/07,Antrim,36791.0,0.0,8984.0,8352.0,0.0,0.0,19414.0,0.0,0.0,51059.0
1,2006/07,Ards,48673.0,0.0,5267.0,6661.0,0.0,0.0,36745.0,0.0,0.0,75767.0
2,2006/07,Armagh,29198.0,0.0,6330.0,3878.0,0.0,0.0,18989.0,0.0,0.0,56785.0
3,2006/07,Ballymena,38269.0,0.0,5520.0,4612.0,0.0,0.0,28137.0,0.0,0.0,61447.0
4,2006/07,Ballymoney,15923.0,0.0,2152.0,1341.0,0.0,0.0,12373.0,0.0,0.0,29392.0


In [96]:
# household (hh) waste dataset
hh_df = df[['fin_year','area','hh_total_waste','hh_reuse','hh_dry_recycle','hh_compost','hh_landfill','num_households','population']]
hh_df.head()

,fin_year,area,hh_total_waste,hh_reuse,hh_dry_recycle,hh_compost,hh_landfill,num_households,population
0,2006/07,Antrim,32732.0,0.0,7069.0,8352.0,17268.0,0.0,51059.0
1,2006/07,Ards,45592.0,0.0,4944.0,6661.0,33986.0,0.0,75767.0
2,2006/07,Armagh,25825.0,0.0,4599.0,3878.0,17346.0,0.0,56785.0
3,2006/07,Ballymena,35989.0,0.0,5520.0,4612.0,25856.0,0.0,61447.0
4,2006/07,Ballymoney,14115.0,0.0,2152.0,1341.0,10621.0,0.0,29392.0


In [97]:
# PREPARE DATASETS

# prepare hh waste
hh_df['hh_recycle_rate'] = (hh_df['hh_reuse'] + hh_df['hh_dry_recycle'] + hh_df['hh_compost']) / hh_df['hh_total_waste'] * 100
hh_df['avg_landfill_waste_per_house'] = hh_df['hh_landfill'] / hh_df['num_households']

# prepare la waste
la_df['la_recycle_rate'] = (la_df['la_reuse'] + la_df['la_dry_recycle'] + la_df['la_compost']) / la_df['la_total_waste'] * 100

/var/folders/c8/mp2gy6x91plc9x44ybwrfj5m0000gn/T/ipykernel_10557/2253211256.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hh_df['hh_recycle_rate'] = (hh_df['hh_reuse'] + hh_df['hh_dry_recycle'] + hh_df['hh_compost']) / hh_df['hh_total_waste'] * 100
/var/folders/c8/mp2gy6x91plc9x44ybwrfj5m0000gn/T/ipykernel_10557/2253211256.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hh_df['avg_landfill_waste_per_house'] = hh_df['hh_landfill'] / hh_df['num_households']
/var/folders/c8/mp2gy6x91plc9x44ybwrfj5m00

In [98]:
import json

# PREPARE JSON OF ALL SELECTABLE YEAR VALUES FOR FILTER OPTION
year_list = df['fin_year'].unique().tolist()
year_list.sort(reverse=True)
year_list = [str(year) for year in year_list]

output = []
for year in year_list:
    output.append({
        'label': year,
        'value': year
    })
# output_json = 


with open("output/selectable_fin_years.json", "w") as outfile: 
    json.dump(output, outfile)

In [99]:
# PREPARE OUTPUT FOR KPIs
import json

# KPI 1 - Current Period Recycle rate (%) (whole NI): 
output = {
    'kpi_1_output': {
        'values': {}
    },
    'kpi_2_output': {
        'values': {}
    },
    'kpi_3_output': {
        'values': {}
    },
    'kpi_4_output': {
        'min_values': {},
        'max_values': {}
    },
    'kpi_5_output': {
        'min_values': {},
        'max_values': {}
    },
    'kpi_6_output': {
        'min_values': {},
        'max_values': {}
    }
}
for year in hh_df['fin_year'].unique():
    # KPI 1
    output['kpi_1_output']['values'][year] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_recycle_rate'].mean()

    # KPI 2
    output['kpi_2_output']['values'][year] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_recycle_rate'].mean()

    # KPI 3
    output['kpi_3_output']['values'][year] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['avg_landfill_waste_per_house'].mean()

    # KPI 4 - best + worst performance of KPI 1 above it
    idx_min = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_recycle_rate'].idxmin()
    output['kpi_4_output']['min_values'][year] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')].loc[idx_min]['area']
    idx_max = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_recycle_rate'].idxmax()
    output['kpi_4_output']['max_values'][year] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')].loc[idx_max]['area']

    # KPI 5 - best + worst performance of KPI 2 above it
    idx_min = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_recycle_rate'].idxmin()
    output['kpi_5_output']['min_values'][year] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')].loc[idx_min]['area']
    idx_max = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_recycle_rate'].idxmax()
    output['kpi_5_output']['max_values'][year] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')].loc[idx_max]['area']

    # KPI 6 - best + worst performance of KPI 3 above it
    idx_min = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['avg_landfill_waste_per_house'].idxmin()
    output['kpi_6_output']['min_values'][year] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')].loc[idx_min]['area']
    idx_max = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['avg_landfill_waste_per_house'].idxmax()
    output['kpi_6_output']['max_values'][year] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')].loc[idx_max]['area']


with open("output/kpi_json_file.json", "w") as outfile: 
    json.dump(output, outfile)

In [102]:
# PREPARE OUTPUT FOR CHARTS
import json
output = {
    'chart_1': {},
    'chart_2': {},
    'chart_3': {},
    'chart_4': {},
}

for year in hh_df['fin_year'].unique():
    # CHART 1 - Recycle rate (%) by LA
    output['chart_1'][year] = {}
    output['chart_1'][year]['values'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_recycle_rate'].tolist()
    output['chart_1'][year]['labels'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['area'].tolist()

    # # CHART 2 - Landfill waste per household by LA
    # output['chart_2'][year] = {}
    # output['chart_2'][year]['values'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['avg_landfill_waste_per_house'].tolist()
    # output['chart_2'][year]['labels'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['area'].tolist()
    
    # CHART 2 - Landfill waste by area by LA
    output['chart_2'][year] = {}
    output['chart_2'][year]['values'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['la_landfill'].tolist()
    output['chart_2'][year]['labels'] = la_df[(la_df['fin_year'] == year) & (la_df['area'] != 'Northern Ireland')]['area'].tolist()

    # CHART 2 - Landfill waste per household by LA

    # CHART 3 - Recycle rate (%) by HH
    output['chart_3'][year] = {}
    output['chart_3'][year]['values'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_recycle_rate'].tolist()
    output['chart_3'][year]['labels'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['area'].tolist()

    # # CHART 4 - Landfill waste per household by HH
    # output['chart_4'][year] = {}
    # output['chart_4'][year]['values'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['avg_landfill_waste_per_house'].tolist()
    # output['chart_4'][year]['labels'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['area'].tolist()

    # CHART 4 - Landfill waste by area by HH
    output['chart_4'][year] = {}
    output['chart_4'][year]['values'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['hh_landfill'].tolist()
    output['chart_4'][year]['labels'] = hh_df[(hh_df['fin_year'] == year) & (hh_df['area'] != 'Northern Ireland')]['area'].tolist()


with open("output/chart_json_file.json", "w") as outfile: 
    json.dump(output, outfile)